###  Import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

### Set base URLs and initialize data

In [2]:
BASE_URL = "https://www.dubizzle.com.om"
URL_TEMPLATE = BASE_URL + "/en/properties/properties-for-rent/?page={}"

MAX_PAGES = 119

dubizzle_data = {
    'property_title': [],
    'location': [],
    'number_of_rooms': [],
    'price': [],
    'size': []
}

### Scraping Loop

In [3]:
for page_number in range(1, MAX_PAGES + 1):
    url = URL_TEMPLATE.format(page_number)
    print(f"Processing page {page_number}: {url}")
    
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
    except requests.exceptions.RequestException as error:
        print(f"Failed to retrieve page {page_number}: {error}")
        break

    soup = BeautifulSoup(response.content, 'html.parser')
    listings = soup.find_all('li', attrs={"aria-label": "Listing"})

    if not listings:
        print(f"No listings found on page {page_number}. Continuing...")
        continue

    for listing in listings:
        title_tag = listing.find('h2', class_='_562a2db2')
        title = title_tag.text.strip() if title_tag else None

        price_tag = listing.find('span', class_='ddc1b288')
        price = price_tag.text.strip() if price_tag else None

        location_tag = listing.find('span', class_='f7d5e47e')
        location = location_tag.text.strip() if location_tag else None

        size = None
        area_block = listing.find('span', attrs={'aria-label': 'Area'})
        if area_block:
            size_span = area_block.find('span', class_='_3e1113f0')
            size = size_span.text.strip() if size_span else None

        number_of_rooms = None
        beds_block = listing.find('span', attrs={'aria-label': 'Beds'})
        if beds_block:
            beds_span = beds_block.find('span', class_='_3e1113f0')
            number_of_rooms = beds_span.text.strip() if beds_span else None

        dubizzle_data['property_title'].append(title)
        dubizzle_data['location'].append(location)
        dubizzle_data['number_of_rooms'].append(number_of_rooms)
        dubizzle_data['price'].append(price)
        dubizzle_data['size'].append(size)

    time.sleep(1)

Processing page 1: https://www.dubizzle.com.om/en/properties/properties-for-rent/?page=1
Processing page 2: https://www.dubizzle.com.om/en/properties/properties-for-rent/?page=2
Processing page 3: https://www.dubizzle.com.om/en/properties/properties-for-rent/?page=3
Processing page 4: https://www.dubizzle.com.om/en/properties/properties-for-rent/?page=4
Processing page 5: https://www.dubizzle.com.om/en/properties/properties-for-rent/?page=5
Processing page 6: https://www.dubizzle.com.om/en/properties/properties-for-rent/?page=6
Processing page 7: https://www.dubizzle.com.om/en/properties/properties-for-rent/?page=7
Processing page 8: https://www.dubizzle.com.om/en/properties/properties-for-rent/?page=8
Processing page 9: https://www.dubizzle.com.om/en/properties/properties-for-rent/?page=9
Processing page 10: https://www.dubizzle.com.om/en/properties/properties-for-rent/?page=10
Processing page 11: https://www.dubizzle.com.om/en/properties/properties-for-rent/?page=11
Processing page 1

###  Convert to DataFrame

In [4]:
df_dubizzle = pd.DataFrame(dubizzle_data)
df_dubizzle.head(10)

,property_title,location,number_of_rooms,price,size
0,ADV905*4BHK Villa for rent in Madinat Illam in...,"Qurum, Muscat•",4,OMR 750,300 SQM
1,ADV906**4BHK Villa in a comples in shatti qurum,"Qurum, Muscat•",4,"OMR 1,900",300 SQM
2,ADC507*** Office Space in Azaiba – 440 sqm for...,"Azaiba, Muscat•",None,"OMR 2,310",440 SQM
3,*ADV705** 3+1 BHK Villa for Rent in Bousher –A...,"Bosher, Muscat•",3,OMR 650,350 SQM
4,4 BR + Maid’s Room Spacious Well-Designed Vill...,"Azaiba, Muscat•",4,OMR 750,439 SQM
5,2 BR Beautiful Apartment with Panoramic Views ...,"Muscat Hills, Muscat•",2,OMR 430,119 SQM
6,"2 BR Fully Furnished Apartment in Juman Two, A...","The Wave (Almouj), Muscat•",2,"OMR 1,250",109 SQM
7,2 BR + 1 Bedroom Brand New Apartment in Shatti...,"Qurum, Muscat•",2,OMR 525,119 SQM
8,1 BR + 1 Bedroom Brand New Apartment in Shatti...,"Qurum, Muscat•",1,OMR 400,89 SQM
9,ADV905*4BHK Villa for rent in Madinat Illam in...,"Qurum, Muscat•",4,OMR 750,300 SQM


### Save as CSV

In [5]:
df_dubizzle.to_csv('dubizzle_rent_listings.csv', index=False, encoding='utf-8-sig')
print(" Data saved to 'dubizzle_rent_listings.csv'")

 Data saved to 'dubizzle_rent_listings.csv'


In [6]:
df_dubizzle.shape

(5355, 5)